In [22]:
from random import randint
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
 
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(1, n_unique-1) for _ in range(length)]
 
# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        
        # define target sequence
        target = source[:n_out]
        target.reverse()
        
        # create padded input target sequence
        target_in = [0] + target[:-1]
#         print('target:',target)
#         print('target-1:',target[:-1])
#         print('target_in:',target_in)
        
        # encode
        src_encoded = to_categorical(source, num_classes=cardinality)
        tar_encoded = to_categorical(target, num_classes=cardinality)
        tar2_encoded = to_categorical(target_in, num_classes=cardinality)
        
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return array(X1), array(X2), array(y)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [14]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [12]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model
 
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return array(output)

In [15]:
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [27]:
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3
# generate a single source and target sequence
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)

In [28]:
print(X1.shape, X2.shape, y.shape)
print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

(100000, 6, 51) (100000, 3, 51) (100000, 3, 51)
X1=[9, 27, 47, 19, 20, 16], X2=[0, 47, 27], y=[47, 27, 9]


In [29]:
train.fit([X1, X2], y, epochs=1)

Epoch 1/1
100000/100000 [==============================] - 40s 399us/step - loss: 0.3923 - acc: 0.8670


In [32]:
from numpy import array_equal

In [33]:
total, correct = 100, 0
for _ in range(total):
    X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
    target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
    if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
    # spot check some examples
    
for _ in range(10):
    X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
    target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
    print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

Accuracy: 100.00%
X=[27, 4, 3, 21, 30, 3] y=[3, 4, 27], yhat=[3, 4, 27]
X=[30, 1, 12, 27, 22, 26] y=[12, 1, 30], yhat=[12, 1, 30]
X=[12, 3, 2, 16, 10, 32] y=[2, 3, 12], yhat=[2, 3, 12]
X=[19, 34, 20, 12, 28, 21] y=[20, 34, 19], yhat=[20, 34, 19]
X=[44, 6, 21, 9, 13, 1] y=[21, 6, 44], yhat=[21, 6, 44]
X=[27, 18, 33, 4, 11, 20] y=[33, 18, 27], yhat=[33, 18, 27]
X=[40, 40, 35, 43, 40, 23] y=[35, 40, 40], yhat=[35, 40, 40]
X=[28, 22, 1, 8, 47, 27] y=[1, 22, 28], yhat=[1, 22, 28]
X=[17, 46, 15, 20, 30, 39] y=[15, 46, 17], yhat=[15, 46, 17]
X=[33, 36, 36, 38, 26, 47] y=[36, 36, 33], yhat=[36, 36, 33]
